In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [3]:
import retro

import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import random
import os

#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using plaidml.keras.backend backend.


In [4]:
#path variables
#game_url = "chrome://dino"
#chrome_driver_path = "/usr/local/bin/chromedriver"
#chrome_driver_path = "./chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
#init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

#get image from canvas
#getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
#return canvasRunner.toDataURL().substring(22)"

In [5]:
class Game:
    def __init__(self,env_name):
        self.env = retro.make(game=env_name)
    def restart(self):
        obs = self.env.reset()
        return obs
    def render(self):
        self.env.render()
        
    def Run(self):
        act = [0,0,0,0,0,0,0,1,0]
        return act
    def Stop(self):
        act = [0,0,0,0,0,0,1,1,0]
        return act
    def Jump(self):
        act = [0,0,0,0,0,0,0,1,1]
        return act

    def get_score(self, act):
        obs, rew, done, info = self.env.step(act)
        
        return obs, rew, done, info
    
    def get_action_sample(self):
        act = self.env.action_space.sample()
        return act
    
    def end(self):
        self.env.close()

In [6]:
class clown_agent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game
        
    def Run(self):
        self._game.Run()
    def Jump(self):
        self._game.Jump()
    def Stop(self):
        self._game.Stop()

In [7]:
class Game_state_:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        
        image, reward, is_over, info = self._game.get_score(actions)
        
        self._game.render()
        
        reward_1 = 0.2 
        
        image_ap = process_img(image)
        
        actions_df.loc[len(actions_df)] = actions[8] # storing actions in a dataframe
        
        self._display.send(image_ap) #display the image on screen
        
        if reward > 50:
            reward_1 = reward_1 + 8
        elif actions[8]==1:
            reward_1 = reward_1 - 0.2
        
        #delay = 200
        score_base = 8180
        
        if info['dead'] < 100:
            score = info['score'] - score_base
            reward_1 = -10
            is_over = True
            scores_df.loc[len(scores_df)] = score # log the score when game is over
            self._game.restart()
        
        
        '''if t % delay == 0:
            if t % (2*delay) == 0 or info['lives']<2:
                tt[0] = info['bonus']
                if tt[0]-tt[1] == 0:
                    score = info['score'] - score_base
                    reward_1 = -10
                    is_over = True
                    scores_df.loc[len(scores_df)] = score # log the score when game is over
                    self._game.restart()
                    tt[0]=0
                    tt[1]=1

            elif t % (2*delay) == delay:
                tt[1] = info['bonus']
                if tt[0]-tt[1] == 0 or info['lives']<2:
                    score = info['score'] - score_base
                    reward_1 = -10
                    is_over = True
                    scores_df.loc[len(scores_df)] = score # log the score when game is over
                    self._game.restart()
                    tt[0]=0
                    tt[1]=1'''

        
        return image_ap, reward_1, is_over, info #return the Experience tuple

In [8]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[35:135, 30:150] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (150, 150)) 
        cv2.imshow(window_title, imS)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [9]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(q_value_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [10]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
Key_num = 9
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 1000. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [11]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [12]:
'''Call only once to init file structure
'''
#init_cache()

'Call only once to init file structure\n'

In [13]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  
    #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [14]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(Key_num)
    #do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
        
    
    x_t, r_0, terminal, info = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 9999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros(Key_num) # action at t
        a_t_fill = [0,0,0,0,0,0,0,1,0]
        #run, jump
        
        
        #choose an action epsilon greedy
        
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                #action_index = random.randrange(ACTIONS)
                action_index = random.choice([0,1])
                a_t_all = [[0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1,1]]
                #a_t = random.choice([[0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1,1]])
                a_t = a_t_all[action_index]

            else: # predict the output
                q = model.predict(s_t)       # input a stack of 4 images, get the prediction
                print(q)
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q
                a_t_all = [[0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1,1]]
                a_t = a_t_all[action_index]       # run, jump
               
        terminal_transfer = False
        
        for i in range(0, 5):
            A1, A2, A3, A4 = game_state.get_state(a_t_fill)
            if A3 == True:
                terminal_transfer = True

                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal, info = game_state.get_state(a_t)
        
        if terminal == False:
            terminal = terminal_transfer
            
        #if a_t[8]==1:
        #    for ii in range(0, 30):
        #        x_t1, r_t, terminal, info = game_state.get_state([0,0,0,0,0,0,0,1,1])

            
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                 
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 100 == 0:
            print("Now we save model")
            #game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            #game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [15]:
'''import retro

def main():
    
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    
    game.restart()
    
    while True:
        obs, rew, done, info = game.get_score(game.Jump())
        game.render()
        if info['lives']<3:
            game.restart()
    game.end()'''

"import retro\n\ndef main():\n    \n    env_name = 'CircusCharlie-Nes'\n    game = Game(env_name)\n    \n    game.restart()\n    \n    while True:\n        obs, rew, done, info = game.get_score(game.Jump())\n        game.render()\n        if info['lives']<3:\n            game.restart()\n    game.end()"

In [16]:
'''import retro

#main function
def main(observe=True):
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    game.restart()
    
    clown = clown_agent(game)
    game_state = Game_state_(clown,game)  
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.restart()
'''

"import retro\n\n#main function\ndef main(observe=True):\n    env_name = 'CircusCharlie-Nes'\n    game = Game(env_name)\n    game.restart()\n    \n    clown = clown_agent(game)\n    game_state = Game_state_(clown,game)  \n    model = buildmodel()\n    try:\n        trainNetwork(model,game_state,observe=observe)\n    except StopIteration:\n        game.restart()\n"

In [17]:
import retro

#main function
def main(observe=False):
    env_name = 'CircusCharlie-Nes'
    game = Game(env_name)
    game.restart()
    
    clown = clown_agent(game)
    game_state = Game_state_(clown,game)
    model = buildmodel()
    
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [ ]:
if __name__ == "__main__":
    main()

TIMESTEP 90800 / STATE explore / EPSILON 0.010290798999872196 / ACTION 0 / REWARD 0.2 / Q_MAX  14.199382 / Loss  0.25859710574150085
[[14.718991 14.695465]]
fps: 0.36001135404155554
TIMESTEP 90801 / STATE explore / EPSILON 0.010289799999872196 / ACTION 0 / REWARD 0.2 / Q_MAX  14.247369 / Loss  0.09985947608947754
[[14.422144 14.70145 ]]
fps: 3.631979189938571
TIMESTEP 90802 / STATE explore / EPSILON 0.010288800999872196 / ACTION 1 / REWARD 0.0 / Q_MAX  14.049167 / Loss  1.348071575164795
[[14.030674 14.373833]]
fps: 3.5400885548809
TIMESTEP 90803 / STATE explore / EPSILON 0.010287801999872196 / ACTION 1 / REWARD 0.0 / Q_MAX  14.354919 / Loss  0.3150983452796936
[[15.052415 14.979464]]
fps: 3.5048742130481507
TIMESTEP 90804 / STATE explore / EPSILON 0.010286802999872197 / ACTION 0 / REWARD 0.2 / Q_MAX  14.914672 / Loss  6.394308567047119
[[14.20164 14.18495]]
fps: 3.4616322302965776
TIMESTEP 90805 / STATE explore / EPSILON 0.010285803999872197 / ACTION 0 / REWARD 0.2 / Q_MAX  14.1496725

fps: 3.5836500341763498
TIMESTEP 90846 / STATE explore / EPSILON 0.010244844999872205 / ACTION 1 / REWARD 0.0 / Q_MAX  15.258469 / Loss  0.11579997837543488
[[2.4896429 3.2532594]]
fps: 3.46030426030426
TIMESTEP 90847 / STATE explore / EPSILON 0.010243845999872206 / ACTION 1 / REWARD 0.0 / Q_MAX  14.607589 / Loss  0.2983451783657074
[[-0.27720857  1.4248879 ]]
fps: 3.5272869779043345
TIMESTEP 90848 / STATE explore / EPSILON 0.010242846999872206 / ACTION 1 / REWARD 0.0 / Q_MAX  15.064971 / Loss  0.1533220261335373
[[-2.623389  -1.8360355]]
fps: 3.455161937650133
TIMESTEP 90849 / STATE explore / EPSILON 0.010241847999872206 / ACTION 1 / REWARD 0.0 / Q_MAX  14.816483 / Loss  0.17210832238197327
[[14.8478565 14.385069 ]]
fps: 3.34277275415266
TIMESTEP 90850 / STATE explore / EPSILON 0.010240848999872206 / ACTION 0 / REWARD 0.2 / Q_MAX  1.730381 / Loss  0.5131662487983704
[[15.5571575 15.012563 ]]
fps: 3.4580048576747466
TIMESTEP 90851 / STATE explore / EPSILON 0.010239849999872206 / ACTION